# Introduction to Machine learning - Classification and Statistics
#### Machine Learning - the science of getting computers to act without being explicitly programmed

MLlib is Spark’s machine learning (ML) library. Its goal is to make practical machine learning scalable and easy. It consists of common learning algorithms and utilities, including classification, regression, clustering, collaborative filtering (this example!), dimensionality reduction, as well as lower-level optimization primitives and higher-level pipeline APIs.

It divides into two packages:
- spark.mllib contains the original API built on top of RDDs.
- spark.ml provides higher-level API built on top of DataFrames for constructing ML pipelines.


Using spark.ml is recommended because with DataFrames the API is more versatile and flexible. But we will keep supporting spark.mllib along with the development of spark.ml. Users should be comfortable using spark.mllib features and expect more features coming.

http://spark.apache.org/docs/latest/mllib-guide.html
<img src='https://raw.githubusercontent.com/bradenrc/Spark_POT/master/Modules/MachineLearning/Classification/titanic.jpg' width="70%" height="70%"></img>
With Spark, we can easily describe data and use it to make predictions.  We'll be using the famous Titanic data set from Kaggle (https://www.kaggle.com/c/titanic/data) and the machine learning package in Spark to do just that.
## Access your data

In [17]:
!wget https://raw.githubusercontent.com/bradenrc/Spark_POT/master/Modules/MachineLearning/Classification/train.csv -N

--2016-06-28 16:58:23--  https://raw.githubusercontent.com/bradenrc/Spark_POT/master/Modules/MachineLearning/Classification/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58625 (57K) [text/plain]
Last-modified header missing -- time-stamps turned off.
--2016-06-28 16:58:23--  https://raw.githubusercontent.com/bradenrc/Spark_POT/master/Modules/MachineLearning/Classification/train.csv
Reusing existing connection to raw.githubusercontent.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 58625 (57K) [text/plain]
Saving to: 'train.csv'

100%[======================================>] 58,625      --.-K/s   in 0.004s  

2016-06-28 16:58:23 (15.9 MB/s) - 'train.csv' saved [58625/58625]



## Data processing
Once we have the data, all of the processing is done in memory.  Here, we're formatting the data, removing columns, dropping rows with insufficient data, creating a DataFrame, and creating columns using user defined functions.

In [18]:
from pyspark.sql import SQLContext,Row
from pyspark.sql.functions import lit

loadTitanicData = sc.textFile("train.csv")
header = loadTitanicData.first()
loadTitanicData = loadTitanicData.filter(lambda l: l != header).\
                                map(lambda l: l.split(",")).\
                                map(lambda l: [l[1],l[2],l[4],l[5],l[6],l[7],l[9],l[11]]).\
                                filter(lambda l: len(l[3]) > 0 and len(l[7]) > 0).\
                                map(lambda l: Row(survived=int(l[0]),\
                                    classRank=int(l[1]),\
                                    sex=l[2],\
                                    age=float(l[3]),\
                                    sibSpou=int(l[4]),\
                                    parChi=int(l[5]),\
                                    fare=float(l[6]),\
                                    embarked=l[7]))

sqlContext = SQLContext(sc)
titanicDf = sqlContext.createDataFrame(loadTitanicData)

from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import IntegerType
isCherb = UserDefinedFunction(lambda x: 1 if x == 'C' else 0, IntegerType())
isQueen = UserDefinedFunction(lambda x: 1 if x == 'Q' else 0, IntegerType())
isSouth = UserDefinedFunction(lambda x: 1 if x == 'S' else 0, IntegerType())
isMale = UserDefinedFunction(lambda x: 1 if x == 'male' else 0, IntegerType())
isFemale = UserDefinedFunction(lambda x: 1 if x == 'female' else 0, IntegerType())
titanicDf = titanicDf.withColumn("cherbourg",isCherb(titanicDf.embarked)).\
                    withColumn("queenstown",isQueen(titanicDf.embarked)).\
                    withColumn("southampton",isSouth(titanicDf.embarked)).\
                    withColumn("male",isMale(titanicDf.sex)).\
                    withColumn("female",isFemale(titanicDf.sex))

titanicDf = titanicDf.drop("sex").drop("embarked")
titanicDf.toPandas().head()

,age,classRank,fare,parChi,sibSpou,survived,cherbourg,queenstown,southampton,male,female
0,22,3,7.2500,0,1,0,0,0,1,1,0
1,38,1,71.2833,0,1,1,1,0,0,0,1
2,26,3,7.9250,0,0,1,0,0,1,0,1
3,35,1,53.1000,0,1,1,0,0,1,0,1
4,35,3,8.0500,0,0,0,0,0,1,1,0


## Gaining insight
### Pearson Correlation
Now that our data is formatted, we can start to do some basic statistics.  Let's look at what features correlate with surviving the titanic crash.
# ADD GRAPH

In [19]:
for col in titanicDf.columns:
    print col + " " + str(titanicDf.corr('survived',col))

age -0.0824458680434
classRank -0.356461588445
fare 0.266099600477
parChi 0.0952652942869
sibSpou -0.0155230236317
survived 1.0
cherbourg 0.195672717021
queenstown -0.0489660937057
southampton -0.159015410677
male -0.536761623349
female 0.536761623349


It doesn't look like age had as much impact as we would have guessed.  Let's try to find to correlation after accounting for gender:

In [20]:
maleTitanicDf = titanicDf.filter(titanicDf.male == 1)
print "male age " + str(maleTitanicDf.corr('survived','age'))
femaleTitanicDf = titanicDf.filter(titanicDf.female == 1)
print "female age " + str(femaleTitanicDf.corr('survived','age'))

male age -0.119617523233
female age 0.110711430946


### Chi Squared Hypothesis Testing
Now that we've seen the correlation, we can double check if they are statistically significant with a chi-squared test:
# ADD GRAPH

In [21]:
from pyspark.mllib.regression import LabeledPoint 
from pyspark.mllib.stat import Statistics

labRDD = titanicDf.map(lambda l: LabeledPoint(l.survived, [l.classRank,l.age,l.sibSpou,l.parChi,l.fare,\
                                                           l.cherbourg,l.queenstown,l.southampton,l.male,l.female]))
features = ["classRank","age","sibSpou","parChi","fare","cherbourg","queenstown","southampton","male","female"]
goodnessOfFitTestResult = Statistics.chiSqTest(labRDD)
count = 0
for result in goodnessOfFitTestResult:
    #print result.pValue
    print features[count] + " " + str(result.pValue)
    count = count + 1

classRank 0.0
age 0.102094296376
sibSpou 0.000429074888302
parChi 6.68106006505e-05
fare 4.74847664522e-08
cherbourg 1.77768084808e-07
queenstown 0.191355954976
southampton 2.20492079113e-05
male 0.0
female 0.0


## Classification Machine Learning
We can use observed data to make predictions on guests' survival.  First, we form our data into a usable format, split it into a training set and a test set, and finally, create predictive models.

In [22]:
from pyspark.mllib.linalg import Vectors
titanicDf = titanicDf.map(lambda l: Row(label=float(l.survived),features=\
                                       Vectors.dense([l.age,float(l.classRank),l.fare,float(l.parChi),float(l.sibSpou),\
                                       float(l.cherbourg),float(l.queenstown),float(l.southampton),\
                                       float(l.male),float(l.female)]))).toDF()
testDf, trainDf = titanicDf.randomSplit([.15,.85],1)

print testDf.take(2)
print
print trainDf.take(2)

[Row(features=DenseVector([22.0, 3.0, 7.25, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0]), label=0.0), Row(features=DenseVector([26.0, 3.0, 7.925, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]), label=1.0)]

[Row(features=DenseVector([38.0, 1.0, 71.2833, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0]), label=1.0), Row(features=DenseVector([35.0, 1.0, 53.1, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0]), label=1.0)]


### Logistic Regression Model:
Logistic regression measures the relationship between the categorical dependent variable and one or more independent variables by estimating probabilities.

In [30]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()
lrModel = lr.fit(trainDf)

lrPred = lrModel.transform(testDf)
print "Error:"
print lrPred.map(lambda line: (line.label - line.prediction)**2).mean()
print "Coefficients:"
print lrModel.coefficients

Error:
0.171171171171
Coefficients:
[-0.0158018334818,-0.591343562615,0.00251736383054,-0.026419046642,-0.119469664559,0.29231409134,-0.38845628511,-0.160703792199,-0.900702156103,0.900703796042]


### Decision Tree Model:
Decision tree learning uses a decision tree as a predictive model which maps observations about an item to conclusions about the item's target value.
<img src='https://raw.githubusercontent.com/bradenrc/Spark_POT/master/Modules/MachineLearning/Classification/CART_tree_titanic_survivors.png' width="50%" height="50%"></img>

In [38]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer

labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(trainDf)
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(trainDf)

dtc = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")
dtcPipeline = Pipeline(stages=[labelIndexer, featureIndexer, dtc])

dtcModel = dtcPipeline.fit(trainDf)
dtcPred = dtcModel.transform(testDf)

print "Error:"
print dtcPred.map(lambda line: (line.label - line.prediction)**2).mean()
print "toDebugString is coming in 2.0, so you can see the entire model."

Error:
0.207207207207
toDebugString is coming in 2.0, so you can see the entire model.


### Random Forest Model:
Random forests are ensembles of decision trees. They combine many decision trees in order to reduce the risk of overfitting.

In [39]:
from pyspark.ml.classification import RandomForestClassifier

rfc = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")
rfcPipeline = Pipeline(stages=[labelIndexer, featureIndexer, rfc])

rfcModel = rfcPipeline.fit(trainDf)
rfcPred = rfcModel.transform(testDf)

print "Error:"
print rfcPred.map(lambda line: (line.label - line.prediction)**2).mean()

Error:
0.198198198198


### Gradient Boosted Tree:
Gradient-Boosted Trees (GBTs) are ensembles of decision trees. GBTs iteratively train decision trees in order to minimize a loss function.

In [40]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")
gbtPipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt])

gbtModel = gbtPipeline.fit(trainDf)
gbtPred = gbtModel.transform(testDf)

print "Error:"
print gbtPred.map(lambda line: (line.label - line.prediction)**2).mean()

Error:
0.234234234234


#### In this case, the logistic regression model was the most accurate.  
Finally.... the ultimate test... would YOU survive the Titanic crash?

In [41]:
#age,classRank,fare,parChi,sibSpou,cherbourg,queenstown,southampton,male,female
userInput = sc.parallelize([Row(features=Vectors.dense([27.0, 3.0,50.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0]))]).toDF()
lrModel.transform(userInput).take(1)

[Row(features=DenseVector([27.0, 3.0, 50.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0]), rawPrediction=DenseVector([1.6247, -1.6247]), probability=DenseVector([0.8354, 0.1646]), prediction=0.0)]